In [1]:
import os, sys    # 다른 폴더에 있는것들 가져오기 위함

module_paths = [
    'C:\\Users\\ledu2\\NLP\\chatbot_study\\개체명 인식 모델\\utils'
]

for path in module_paths:
    abs_path = os.path.abspath(path)
    if abs_path not in sys.path:
        sys.path.append(abs_path)

In [2]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from Preprocess import Preprocess

In [3]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [4]:
# 전처리 객체 생성
p = Preprocess(word2index_dic = '../../train_tools/dict/chatbot_dict.bin',
              userdic = '../../utils/user_dic.tsv')

In [5]:
# 학습용 말뭉치 데이터 불러오기
corpus = read_file('ner_train.txt')

In [6]:
# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)

In [8]:
print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 : ", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 : ", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 61999
0번째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 :  168
샘플 단어 시퀀스 평균 길이 :  8.796238649010467


In [9]:
# 토크나이저 정의
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보 lower = False 소문자 변환 X
tag_tokenizer.fit_on_texts(tags)

In [11]:
# 단어 사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 : ", tag_size)
print("단어 사전 크기 : ", vocab_size)

BIO 태그 사전 크기 :  10
단어 사전 크기 :  17751


In [29]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [13]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding = 'post', maxlen = max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding = 'post', maxlen = max_len)

In [14]:
# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = .2, random_state = 1234)

In [15]:
# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

In [16]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_train.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (49599, 40)
학습 샘플 레이블 형상 :  (49599, 40, 10)
테스트 샘플 시퀀스 형상 :  (49599, 40)
테스트 샘플 레이블 형상 :  (12400, 40, 10)


# 모델 정의(Bi-LSTM)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [19]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 30, input_length = max_len, mask_zero = True))
model.add(Bidirectional(LSTM(200, return_sequences = True, dropout = 0.50, recurrent_dropout = 0.25)))
model.add(TimeDistributed(Dense(tag_size, activation = 'softmax')))
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.01), metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size = 128, epochs = 10)

Epoch 1/10
388/388 [==============================] - 195s 484ms/step - loss: 0.1267 - accuracy: 0.9647
Epoch 2/10
388/388 [==============================] - 193s 498ms/step - loss: 0.0394 - accuracy: 0.9874
Epoch 3/10
388/388 [==============================] - 197s 508ms/step - loss: 0.0259 - accuracy: 0.9915
Epoch 4/10
388/388 [==============================] - 201s 519ms/step - loss: 0.0204 - accuracy: 0.9933
Epoch 5/10
388/388 [==============================] - 228s 587ms/step - loss: 0.0177 - accuracy: 0.9941
Epoch 6/10
388/388 [==============================] - 228s 587ms/step - loss: 0.0155 - accuracy: 0.9947
Epoch 7/10
388/388 [==============================] - 231s 596ms/step - loss: 0.0137 - accuracy: 0.9952
Epoch 8/10
388/388 [==============================] - 229s 589ms/step - loss: 0.0126 - accuracy: 0.9956
Epoch 9/10
388/388 [==============================] - 230s 594ms/step - loss: 0.0121 - accuracy: 0.9958
Epoch 10/10
388/388 [==============================] - 187s 483m

In [21]:
print("평과 결과 : ", model.evaluate(x_test, y_test)[1])
model.save('ner_model.h5')

388/388 [==============================] - 30s 76ms/step - loss: 0.0578 - accuracy: 0.9868
평과 결과 :  0.9867639541625977


In [23]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):    #예측값을 index_to_ner를 설정하는 태깅 정보로 변경하는 함수
    result = []
    for sequence in sequences:    # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:    # 시퀀스로부터 예측값을 하나씩 꺼낸다
            pred_index = np.argmax(pred)    # 예를 들어 [0, 0, 1, 0, 0] 이라면 1의 인덱스인 2를 리턴
            temp.append(index_to_ner[pred_index].replace("PAD", "0"))    # 'PAD'는 '0'로 변경
        result.append(temp)
    return result

In [26]:
# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report

In [30]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted)    # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

388/388 [==============================] - 23s 60ms/step


In [31]:
# F1 평가 결과
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

C:\Users\ledu2\AppData\Local\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\ledu2\AppData\Local\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\ledu2\AppData\Local\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\ledu2\AppData\Local\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\ledu2\AppData\Local\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk)

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       303
           _       0.57      0.53      0.55       647
         _DT       1.00      1.00      1.00     13683
       _FOOD       1.00      1.00      1.00     11655
         _LC       0.73      0.60      0.66       314
         _OG       0.58      0.52      0.55       460
         _PS       0.62      0.55      0.59       396
         _TI       0.92      0.75      0.83        61

   micro avg       0.97      0.97      0.97     27519
   macro avg       0.80      0.74      0.77     27519
weighted avg       0.97      0.97      0.97     27519

F1-score: 97.1%
